In [1]:
#!/usr/bin/env python
# coding: utf-8

# ## SOM_CV_data_yrstrain_UKESM
# 
# This notebook trains the SOMs from the input data where there is a given GTD (either era5 or UK-ESM-1-0-LL piControl). From a specified number of folds in the cross-validation, the function trains and plots the SOM and associated trends from the training data. The SOM is then applied to the test dataset. This is done for each specified set of years.
# 
# 
# This notebook looks at varying the number of years in the test dataset with a constant number of years trained over, so that by seeing the change in F1 score over time we can identify the minimum number of years needed to constrain the blocking climatology
# 

import numpy as np
import scipy
import netCDF4
import matplotlib as mpl
mpl.use("Agg", warn=False)
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import sys
import pandas as pd
import os
os.environ["PROJ_LIB"] = "/rds/general/user/kc1116/home/anaconda3/envs/zeus/share/proj"
import matplotlib as mpl
import matplotlib
import matplotlib.colors as colors
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D
import somoclu
import cartopy
import xarray
import xarray as xr
import glob
import math
from mpl_toolkits.basemap import Basemap as bm
from itertools import groupby
from scipy import stats
import cartopy.crs as ccrs
from statsmodels.stats.multitest import (multipletests, fdrcorrection,
                                         fdrcorrection_twostage,
                                         NullDistribution,
                                         local_fdr)
from scipy.stats import spearmanr, kendalltau, pearsonr
from functools import wraps
import errno
import os
import signal
import SOM_trends_funcs as SOM_fn



In [2]:
def sel_JJA_extd(da):
    """
    Selects the JJA extd period +- 5 days.
    """
    da_MJJAS=da.sel(time = np.isin(da['time.month'], np.arange(5,10)))
    da_JJAextdS=da_MJJAS.sel(time=~(((da_MJJAS.time.dt.month == 5) & (da_MJJAS.time.dt.day < 28))))
    da_JJAextd=da_JJAextdS.sel(time=~(((da_JJAextdS.time.dt.month == 9) & (da_JJAextdS.time.dt.day > 4))))
    return da_JJAextd

In [6]:

def save_SOM_data(zg_file, yrst, yrend, season, region, init, neigh, std, ep, rad_0, rad_N, rad_cooling, scale_0, scale_N, scale_cooling,
            savefig_title, savefig_trends, n_rows, n_columns, samp, zg_str, colormap_str, suptitle, mult_test_method, #yrst=1979, yrend=2013
                data_yr_reshaped, vmin, vmax, step, lat_str, lon_str, units, mdl, train_input = False, som_yr = None, JJA_days=104, save_SOM_str = False, save_SOM_trends_str = False):
    """
    Save the data for all the climate models
    """
    #Get the prepared data
    #issue with prepData function so load information beforehand
    #lats, lons, dates, nt_yr, nr_lat, nr_lon, m = prepData(zg_file, yrst, yrend, "JJA", lats_arr, lons_arr, zg_str, grid_res, lat_str, lon_str)
    lats, lons, dates = zg_file[lat_str], zg_file[lon_str], data_yr_reshaped['time']
    yrst, yrend = int(data_yr_reshaped['time.year'].min()), int(data_yr_reshaped['time.year'].max())
    nr_lat, nr_lon = len(lats), len(lons)
    m = bm(projection='cyl',llcrnrlat=lats[0],urcrnrlat=lats[-1],llcrnrlon=lons[0],urcrnrlon=lons[-1],resolution='l')
    print(f"save_SOM data_yr_reshaped.shape = {data_yr_reshaped.shape}")
    #Train the SOM if not inputting an already trained SOM pattern
    if train_input == False:
        print("training")
        som_yr = somoclu.Somoclu(n_columns, n_rows, maptype="planar",compactsupport=True,initialization=f"{init}", neighborhood=f"{neigh}", std_coeff=std)
        som_yr.train(data_yr_reshaped,epochs=ep,radius0=rad_0,radiusN=rad_N,radiuscooling=f"{rad_cooling}",
                 scale0=scale_0,scaleN=scale_N,scalecooling=f"{scale_cooling}")
        bmus=som_yr.bmus
    else:
        #already have som_yr but need to calculate bmus
        codebook_da = xarray.DataArray(som_yr.codebook, name = "codebook", dims = ("row", "col", "latlon"))
        codebook_da_reshaped = codebook_da.values.reshape(n_rows*n_columns, data_yr_reshaped.shape[1])
        codebook_da_reshaped_xr = xr.DataArray(codebook_da_reshaped, name = f"codebook_reshaped")
        codebook_da_reshaped_xr = codebook_da_reshaped_xr.rename(dim_0="rowcol").rename(dim_1="latlon_flat")
        bmus = SOM_fn.Identify_BMU_from_codebook(codebook_da_reshaped_xr, data_yr_reshaped, n_rows, n_columns)
        
    #Plot the trained SOM nodes 
    g, axes = plt.subplots(nrows=n_rows, ncols=n_columns, figsize=(10,5))
    g.subplots_adjust(hspace=0.2, wspace=0.2)
    #Store the frequency data in a list
    freq_list = []
    for i in range(n_rows): 
        for j in range(n_columns):
            try:
                ax = axes[i][j]
                node = som_yr.codebook[i][j]
            except: #doesn't work when not indexed, when one of n_rows or n_columns = 1
                if n_rows > n_columns:
                    ax = axes[i]
                    node = som_yr.codebook[j]
                else:
                    ax = axes[i]
                    node = som_yr.codebook[i]
            #print(f"node = {node}")
            #print(f"node.shape = {node.shape}")
            node_orig = node.reshape(nr_lat,nr_lon)            
            
            SOM_fn.plot_field(m, node_orig, lats, lons, -330, 330, 30, ncols = n_columns, nrows = n_rows,  #for pv use -330/250, 330/250, 30/250 as levels
                       ax=ax, grid=False, cmap=plt.get_cmap(colormap_str), fig=g, row=i, col=j)
            #Pattern frequencies
            if train_input == False:
                freq = sum([1 for pat in som_yr.bmus if pat.tolist() == [j,i]])/float(len(som_yr.bmus))*100 #note the flipped indices
            else:
                #print(f"pat = {[j,i]}")
                freq = sum([1 for pat in bmus if pat == [j,i]])/float(len(bmus))*100 #note the flipped indices
                #print(f"freq = {freq}")
            freq_list.append(freq)
            props = dict(boxstyle='round', facecolor='white', alpha=0.8)
            ax.text(0.05, 0.95, str(round(freq,1))+"%", transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)
            #ax1.ticklabel_format(style="plain")      
    suptitle=f"zg LTDM anom {mdl} {yrst}-{yrend} JJA ({units})"            
    g.suptitle(f"{suptitle}", fontsize=10)    
    ax.ticklabel_format(style="plain")      
    if save_SOM_str != False:
        print(f"save_SOM_str = {save_SOM_str}")
        g.savefig(save_SOM_str, bbox_inches="tight", dpi = 300)
    plt.close();
        
    da_xr_trend, all_occur = saveSOMTrends(som_yr, dates, n_rows, n_columns, savefig_trends, colormap_str,
                                mult_test_method, mdl, train_input=train_input, bmus=bmus, JJA_days=JJA_days, yrst=yrst, yrend=yrend, samp=samp)#, save_SOM_trends_str=save_SOM_trends_str)
    return da_xr_trend, all_occur, som_yr


#Function to plot the SOM trends
#added values to modify Gerald's function (issues with date formatting)
def saveSOMTrends(som_yr, dates, n_rows, n_columns, savefig_trends, colormap_str, mult_test_method, mdl, train_input=False, bmus=None, JJA_days=104, 
                  yrst=1979, yrend=2019, samp=2, plot=True):
    """
    Plots the trends (occurrence, persistence, max duration) of the SOM patterns generated from the GPH data. 
    
    Parameters
    ----------
    som_yr    : somoclu.train.Somoclu
        The trained SOM (somoclu object)
    seas_yr   : xarray.DataArray
        Data array containing GPH data and information on time, latitude, longitude variables in the selected time range, season and region
        
    Returns
    ---------
    grad_list : list 
        List of the gradients of the pattern occurrence trends computed by linear least-squares regression 
    all_occur : list 
        List of occurrences per day in the trained dataset for each SOM pattern
    """
    if train_input==False:
        #calculate the best matching unit
        #Extract pattern data (occurence, persistence, max duration)
        #Store each pattern data as a binary list of len(dates), i.e. [1,0,0,0,1...,1]
        bmus = som_yr.bmus
    
    global all_occur
    all_occur = [[] for i in range(max(n_rows,n_columns))]
    for a in range(n_rows):
        for b in range(n_columns): 
            curr_occur = []
            for c in bmus:
                if train_input==False:
                    if c.tolist() == [b,a]: #### need to swap the indices here!! #b'a
                        curr_occur.append(1) #This pattern occured on this date
                    else:
                        curr_occur.append(0) #This pattern didn't occur 
                else:
                    if c == [b,a]: #### need to swap the indices here!! #b'a
                        curr_occur.append(1) #This pattern occured on this date
                    else:
                        curr_occur.append(0) #This pattern didn't occur                     
            all_occur[a].append(curr_occur)
    #Loop through each set of circulation occurence data
    da_arr, rownum_arr, colnum_arr = [], [], []
    
   
    
    ##### Plotting - can put in a different function
    #####
    if plot:
        #Plotting axes the extracted circulation data
        if n_columns == 1:
            f, axes1 = plt.subplots(nrows=n_rows, ncols=n_columns+1, figsize=(10,10))
        elif n_rows == 1:
            f, axes1 = plt.subplots(nrows=n_rows+1, ncols=n_columns, figsize=(10,10))
        else:
            f, axes1 = plt.subplots(nrows=n_rows, ncols=n_columns, figsize=(10,10))
        f.subplots_adjust(hspace=0.2, wspace=0.2)
    test_num = 8 #number of p value tests in family of hypotheses (occ, persis, max_dur and event_no * 2)
    #pvalue and gradient matrices for each node
    pval_mat, grad_mat = np.zeros((n_rows,n_columns,test_num)), np.zeros((n_rows,n_columns,test_num))    
    grad_list = []
    for row in range(n_rows):  
        for col in range(n_columns):
            #Need some exception handling for rare cases where any of the patterns doesnt occur in a year
            #NOTE the swapped indices [col][row] here! Doulbe check with Joy's code, see if node number can be confirmed
            #print(f"all_occur[row][col] = {all_occur[row][col]}")
            #all_occur[row][col] is the relevant matrix for investigating case studies
            cum_data = pd.Series(np.array(all_occur[row][col]), index=dates.values)
            #print(f"cum_data = {cum_data}")
            #to generate the years in the dataset, since cum_data.index.year has issues
            #print(f"JJA_days = {JJA_days}")       
            #print(f"cum_data.shape = {cum_data.shape}")
            years_num=int(cum_data.shape[0]/JJA_days)
            #print(f"cum_data.shape[0]/JJA_days = {cum_data.shape[0]}/{JJA_days} = {cum_data.shape[0]/JJA_days}")
            #print(f"years_num = {years_num}")
            years_zeros = np.zeros((years_num*JJA_days))
            if type(yrst) == int or type(yrst) == float:
                arr_gen = np.array([years_zeros[JJA_days*i:JJA_days*(i+1)]+i+yrst for i in range(years_num)
                               ]).reshape(years_num*JJA_days)
            else: #yrst an array
                print("yrst an array")
                arr_gen = np.array([years_zeros[JJA_days*i:JJA_days*(i+1)]+i+int(yrst.values) for i in range(years_num)
                               ]).reshape(years_num*JJA_days)
            #print(f"len(arr_gen) = {len(arr_gen)}")
            sort_to_year = cum_data.groupby(arr_gen).apply(list)
            year = np.array(sort_to_year.keys().tolist()) #[1979, 1980,...]
            occ = [sum(d) for d in sort_to_year]  #[25,40,...]
            # Count streaks 
            streaks = [[sum(1 for i in g) for k,g in groupby(x) if k == 1] for x in sort_to_year]
            streak_count = [[(k, sum(1 for i in g)) for k,g in groupby(sorted(x)) ] for x in streaks]
            
            #Persistence (if statement to avoid division by zero )
            persis = [ sum([x[0]*x[1] for x in year])/sum([x[1] for x in year]) if sum([x[1] for x in year]) != 0 else 0 for year in streak_count ]
            #Max Duration 
            max_dur = [ year[-1][0] if year != [] else 0 for year in streak_count]
            ##want to store occ, persis, max_dur and ev_no in a xarray Dataset/DataArray to save, with the time file showing the years
            ##need to store this for each node (36 different 1d arrays for each model, plus the timestamp)

            def len_iter(items):
                return sum(1 for _ in items)            
            
            def consecutive_one_len(data):
                return len(list(len_iter(run) for val, run in groupby(data) if val))            
            event_no = [consecutive_one_len(d) for d in sort_to_year] 
            
            store = [occ, persis, max_dur, event_no]
            
            da = xarray.DataArray(store, dims = ('SOM_trend_metrics', "year"))
            da['year'] = year
            da['SOM_trend_metrics'] = ['occ', 'persis', 'max_dur', 'event_no']
            da_arr.append(da)
            #store the DataArray and the accompanying row and column number of the SOM pattern            
            rownum_arr.append(row)
            colnum_arr.append(col)
            
            if plot:
                def len_iter(items):
                    return sum(1 for _ in items)            

                def consecutive_one_len(data):
                    return len(list(len_iter(run) for val, run in groupby(data) if val))            
                event_no = [consecutive_one_len(d) for d in sort_to_year]      
                #event_no = [sum(d) for d in sort_to_year]        
                #np.ones((len(sort_to_year)))*sum(sort_to_year)
                #Fitting via Linear Least Squares Regression (With Serial Correlation)
                def fit(m,x,c):  #Simple linear function
                    return m*x + c

                #Occurrence 
                m0, c0, r0, p0, err0 = scipy.stats.linregress(year,occ)
                m1, c1, r1, p1, err1 = scipy.stats.linregress(year[:],occ[:])
                fit_occ = [fit(m0,x,c0) for x in year]
                fit_occ1 = [fit(m1,x,c1) for x in year[:]]
                grad_list.append(m0)

                #Persistence
                m2, c2, r2, p2, err2 = scipy.stats.linregress(year,persis)
                m3, c3, r3, p3, err3 = scipy.stats.linregress(year[:],persis[:])
                fit_per = [fit(m2,x,c2) for x in year]
                fit_per1 = [fit(m3,x,c3) for x in year[:]]

                #Max duration
                m4, c4, r4, p4, err4 = scipy.stats.linregress(year,max_dur)
                m5, c5, r5, p5, err5 = scipy.stats.linregress(year[:],max_dur[:])
                fit_dur = [fit(m4,x,c4) for x in year]
                fit_dur1 = [fit(m5,x,c5) for x in year[:]]

                #Event number
                m6, c6, r6, p6, err6 = scipy.stats.linregress(year,event_no)
                m7, c7, r7, p7, err7 = scipy.stats.linregress(year[:],event_no[:])
                fit_evno = [fit(m6,x,c6) for x in year]
                fit_evno1 = [fit(m7,x,c7) for x in year[:]]
                
                #Plot results
                row, col = int(row), int(col)
                occ, persis, max_dur, event_no = np.array(occ), np.array(persis), np.array(max_dur), np.array(event_no)
                
                axes1[row][col].scatter(year, occ, c='k', s=3, label="Occurrences") #plot occurrences vs years
                axes1[row][col].plot(year, fit_occ, c='k')
                axes1[row][col].plot(year[:], fit_occ1, '--', c='k')

                axes1[row][col].scatter(year, persis, c='b', s=3, label="Persistence") #plot persistence vs years
                axes1[row][col].plot(year, fit_per, c='b')
                axes1[row][col].plot(year[:], fit_per1, '--', c='b')

                axes1[row][col].scatter(year, max_dur, c='r', s=3, label="Max Duration") #plot max duration vs years
                axes1[row][col].plot(year, fit_dur, c='r')
                axes1[row][col].plot(year[:], fit_dur1, '--', c='r')

                axes1[row][col].scatter(year, event_no, c=(0.5,0.5,0.5), s=3, label="Event no") #plot event number vs years
                axes1[row][col].plot(year, fit_evno, c=(0.5,0.5,0.5))
                axes1[row][col].plot(year[:], fit_evno1, '--', c=(0.5,0.5,0.5))         

                if n_rows*n_columns > 5:
                    font_size_text = 8
                    font_size_title = 9 
                else:
                    font_size_text = 11
                    font_size_title = 12                  
                font_size_suptitle = 14
                #corrected p values
                (pval_occ, pval_occ_samp, pval_persis, pval_persis_samp, 
                 pval_max_dur, pval_max_dur_samp, pval_event_no, pval_event_no_samp) = (SOM_fn.lmtrendtest(year, occ), SOM_fn.lmtrendtest(year[:], occ[:]),
                                                                                        SOM_fn.lmtrendtest(year, persis), SOM_fn.lmtrendtest(year[:], persis[:]),
                                                                                        SOM_fn.lmtrendtest(year, max_dur), SOM_fn.lmtrendtest(year[:], max_dur[:]),
                                                                                        SOM_fn.lmtrendtest(year, event_no), SOM_fn.lmtrendtest(year[:], event_no[:]))
                #store the calculated p values in a matrix
                pval_mat[row,col,:] = [pval_occ, pval_occ_samp, pval_persis, pval_persis_samp, pval_max_dur, pval_max_dur_samp, pval_event_no, pval_event_no_samp]
                grad_mat[row,col,:] = [m0,m1,m2,m3,m4,m5,m6,m7]
    
    if plot:
        if mult_test_method == "none":
            print("No multiple hypothesis correction applied")
        elif mult_test_method == "k-FWER":    
            #test for the k-familywise error rate using the Bonferroni correction
            pval_arr = SOM_fn.Bonferroni_kFWER(pval_mat[:,:,:].flatten(), k = 20)
            pval_mat = pval_arr.reshape((n_rows,n_columns,test_num)) 
        else:
            pval_arr = multipletests(pval_mat[:,:,:].flatten(), alpha=0.05, method=mult_test_method, is_sorted=False, returnsorted=False)[1]  
            pval_mat = pval_arr.reshape((n_rows,n_columns,test_num)) 

        for row in range(n_rows):  
            for col in range(n_columns):
                m0,m1,m2,m3,m4,m5,m6,m7 = grad_mat[row,col,:]
                p0,p1,p2,p3,p4,p5,p6,p7 = pval_mat[row,col,:]
                #Legend
                #calculate the corrected p values and create the strings to reduce the bold text
                m0_str, m1_str, m2_str, m3_str, m4_str, m5_str, m6_str, m7_str = (SOM_fn.pval_str(m0, p0), SOM_fn.pval_str(m1, p1), 
                                                                  SOM_fn.pval_str(m2, p2), SOM_fn.pval_str(m3, p3), 
                                                                  SOM_fn.pval_str(m4, p4), SOM_fn.pval_str(m5, p5),
                                                                  SOM_fn.pval_str(m6, p6), SOM_fn.pval_str(m7, p7))
                axes1[row][col].text(0.16, 0.99, "Occurrence", transform=axes1[row][col].transAxes, fontsize=font_size_title, verticalalignment='top', horizontalalignment='center')
                axes1[row][col].text(0.16, 0.93, m0_str,
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center')
                axes1[row][col].text(0.16, 0.87, m1_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center')

                axes1[row][col].text(0.49, 0.99, "Persistence", transform=axes1[row][col].transAxes, fontsize=font_size_title, verticalalignment='top',horizontalalignment='center', color='b')
                axes1[row][col].text(0.49, 0.93, m2_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center', color='b')
                axes1[row][col].text(0.49, 0.87, m3_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center', color='b')

                axes1[row][col].text(0.83, 0.99, "Max duration", transform=axes1[row][col].transAxes, fontsize=font_size_title, verticalalignment='top', horizontalalignment='center', color='r')
                axes1[row][col].text(0.83, 0.93, m4_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center', color='r')
                axes1[row][col].text(0.83, 0.87, m5_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center', color='r')      

                axes1[row][col].text(0.16, 0.82, f"Event no", transform=axes1[row][col].transAxes, fontsize=font_size_title, verticalalignment='top', horizontalalignment='center', color=(0.5,0.5,0.5))
                axes1[row][col].text(0.16, 0.76, m6_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center', color=(0.5,0.5,0.5))
                axes1[row][col].text(0.16, 0.70, m7_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center', color=(0.5,0.5,0.5))               

                axes1[row][col].set_ylim((0,55))            
            f.suptitle(f'Time series of circulation trends \n {mdl} {yrst}-{yrend}', fontsize=14)
        f.savefig(savefig_trends, bbox_inches="tight", dpi=300) 
        plt.close();
    #####        
    #####        
            
    #concatenate the array
    da_xr = xarray.concat(da_arr, dim = ("node"))
    #add a new coordinates to specify the row and column number for each layer
    da_xr = da_xr.assign_coords(
        rownum=('node', rownum_arr))
    da_xr = da_xr.assign_coords(
        colnum=('node', colnum_arr))           
            
    return da_xr, all_occur


#file_zg_str, n_rows, n_columns, yrnum_test, yrnum_train_red, num_yrs_train
def SOM_calc_samp(file_zg_str, n_rows, n_columns, yrnum_test, yrnum_train_red, num_yrs_train):
    """
    Calculate and save the trained arrangement of SOM nodes for a set of subsamples of the data for the purposes of cross-validation 
    """
    file_zg_tot = xarray.open_dataset(file_zg_str).squeeze()
    
    yrst, yrend = int(file_zg_tot['time.year'].min()), int(file_zg_tot['time.year'].max())
    shift_years = 0
    if "era5" in file_zg_str:
        shift_years = 1 # shift the starting year by 1 to skip cross-validation on 1979
    yrst_train_arr = range(1960,2060,10)#[yrst+shift_years+i*yrnum_test for i in range(int((yrend - yrst)/yrnum_test))]   
    
    k = 10#int((yrend - yrst)/yrnum_test)
    #print(f"{k}-fold cv")    
    if "rel_vort" in file_zg_str:
        zg_str = "rel_vort"
        units = "s$^{-1}$"
        if "era5" in file_zg_str:
            data_yr_reshaped_str = "/rds/general/project/carl_phd/live/carl/data/era5/day/rel_vort/500xi_era5_day_1979-2019_1x1_EUR_LTDManom_reshaped.nc"
        else:
            data_yr_reshaped_str = ("/rds/general/project/carl_phd/live/carl/data/cmip6/UKESM1-0-LL/piControl/day/r1i1p1f2/rel_vort/"
            "500xi_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_1960-2060_r180x91_latcorr_EUR_LTDManom_JJAextd_reshaped.nc")
            #("/rds/general/project/carl_phd/live/carl/data/cmip6/UKESM1-0-LL/piControl/day/r1i1p1f2/rel_vort/"
            #"500xi_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_1960-2060_r180x91_latcorr_EUR_LTDManom_JJAextd_reshaped.nc")
    pt_str = "_350pt"
    
    if "vpv" in file_zg_str:
        zg_str = "VPV"
        units = "PVU"
        data_yr_reshaped_str = "/rds/general/project/nowack_graven/live/carl/pv_era5/vpv_1x1_day_150-500hPa_hires_1979-2019_LTDManom_JJAextd_EUR_reshaped.nc"
    if "pv_330pt" in file_zg_str:
        zg_str = "ipv_330pt"
        units = "PVU"
        data_yr_reshaped_str = "/rds/general/project/carl_phd/live/carl/data/era5/day/pv/isentropes/pv_330pt_era5_1979-2019_1x1_EUR_daymean_invlat_LTDManom_JJAextd_reshaped.nc"
    if "pv_350pt" in file_zg_str:
        zg_str = "pv_350pt"
        units = "PVU"
        data_yr_reshaped_str = "/rds/general/user/cmt3718/home/pv_350pt_era5_1979-2019_1x1_tot_3hr_daymean_ydayavg_LTDManom_invlat_JJAextd_reshaped.nc"
    if "psl" in file_zg_str:
        zg_str = "psl"
        units = "Pa"
        data_yr_reshaped_str = ("/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/psl/"
 "psl_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR2_JJAextd_dtrnd_reshaped.nc")     
    if "msl" in file_zg_str:
        zg_str = "msl"
        units = "Pa"
        data_yr_reshaped_str = f"/rds/general/project/nowack_graven/live/carl/era5/mean_sea_level_pressure/mslp_ERA5_{yrst}-{yrend}_EUR_JJAextd_LTDMdaymean_anom_reshaped2.nc"
    if "zg" in file_zg_str:
        zg_str = "z"
        units = "hPa"
        data_yr_reshaped_str = "/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_sort_reshaped.nc"
        #("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/500zg_1x1_1979-2019_JJAextd_LTDManom_nolp_EurAR5_timedtrnd_reshaped.nc")
        #f"/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_{yrst}-{yrend}_EUR_JJAextd2_LTDMdaymean_anom_reshaped.nc"
    if "UKESM" in file_zg_str:
        if "rel_vort" in file_zg_str:
            #data_yr_reshaped_str = "/rds/general/project/carl_phd/live/carl/data/cmip6/UKESM1-0-LL/piControl/day/r1i1p1f2/rel_vort/500xi_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_1960-2060_r180x91_latcorr_EUR_LTDManom_JJAextd_reshaped.nc"
            #print(data_yr_reshaped_str)
           ## print(xr.open_dataset(data_yr_reshaped_str))
            data_yr_reshaped = xr.open_dataset(data_yr_reshaped_str)['UKESM1-0-LL_rel_vort_reshaped']
        if "zg" in file_zg_str:
            zg_str = "zg"
            data_yr_reshaped_str = ("/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/zg/"
                "500zg_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR_JJAextd_dtrnd_reshaped.nc")
            data_yr_reshaped = xr.open_dataset(data_yr_reshaped_str)['data_yr_reshaped']
        if "psl" in file_zg_str:
            zg_str = "psl"
            data_yr_reshaped_str = ("/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/psl/"
                "psl_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR_JJAextd_dtrnd_reshaped.nc")
            data_yr_reshaped = xr.open_dataset(data_yr_reshaped_str)['data_yr_reshaped']
    if "era5" in data_yr_reshaped_str:
        if "pv_350pt" in file_zg_str:
            data_yr_reshaped = xr.open_dataset(data_yr_reshaped_str)[f'era5_ipv_350K_reshaped']
        else:
            data_yr_reshaped = xr.open_dataset(data_yr_reshaped_str)[f'era5_{zg_str}_reshaped']
    file_zg_tot = file_zg_tot[zg_str.split(pt_str)[0]]
    if "rel_vort" not in file_zg_str and "UKESM" not in file_zg_str:
        file_zg_tot = sel_JJA_extd(file_zg_tot)

    lat_str, lon_str = "latitude", "longitude"
    if "era5" in file_zg_str:
        mdl, ens = "era5", "reanal"
        mdl_ens_str = f"{mdl}_{ens}"
        mip = "reanal"
    if "UK" in file_zg_str:
        mdl, ens = "UKESM1-0-LL", "r1i1p1f2"
        mdl_ens_str = f"{mdl}_{ens}"
        mip = "cmip6"
        lat_str, lon_str = "lat", "lon"
    lats, lons = np.arange(30,77), np.arange(-10,41)
    #need to adjust the longitude coordinate if it runs from 0 - 360 E instead of -180 to +180 E
    file_zg_tot=SOM_fn.da_lon_adj(file_zg_tot)
    if "UKESM" in file_zg_str:
        file_zg_tot = file_zg_tot.sel(time = np.isin(file_zg_tot['time.dayofyear'], np.arange(147,245)))
    else:
        JJA_extd_daymonth_xr = xr.open_dataset("/rds/general/project/nowack_graven/live/carl/era5/day_month_1979-2019_JJAextd.nc")['JJA_extd']
        #zg_file['day_month'] = xr.open_dataset("/rds/general/project/nowack_graven/live/carl/era5/day_month_1979-2019_nolp.nc")['day_month']
        day_month = xr.open_dataset("/rds/general/project/nowack_graven/live/carl/era5/day_month_1979-2019_nolp.nc")['day_month']
        file_zg_tot = file_zg_tot.sel(latitude = np.isin(file_zg_tot['latitude'], lats), 
                                  longitude = np.isin(file_zg_tot['longitude'], lons))
        #if "msl" in file_zg_str:
        #    file_zg_tot = file_zg_tot.sel(time = np.isin(day_month, JJA_extd_daymonth_xr))
        #.sel(time = np.isin(file_zg_tot['time.year'], np.arange(yrst_train, yrst_train+yrnum_train+1), invert=True))[zg_str]
    num_days, num_lats, num_lons = file_zg_tot.shape

    vmin, vmax, step = -15, 15, 1.5
    init, neigh, std, ep, rad_0, rad_N, rad_cooling, scale_0, scale_N, scale_cooling = "pca", "gaussian", 0.5, 50, 1, 0, "linear", 0.1, 0.01, "exponential"
    region, season = "Europe", "JJA_extd"
    domain="EUR"
    samp =2 #int((yrst_train_arr[1]-yrst_train_arr[0])/2)
    colormap_str = "seismic"
    suptitle=f"zg LTDM anom {mdl} {yrst}-{yrend} JJA extd ({units})"
    #all cover different variations on the mltiple hypothesis test applied in Horton et al 2015
    mult_test_method = "fdr_bh"#"k-FWER"#"none"#"fdr_bh"

    #cross-validation training period
    for yrst_train in yrst_train_arr[:]:
        print(" --------------------- \n ")
        print(f"train period {yrst_train}-{yrst_train+yrnum_test}")
        #print(f"file_zg_tot = {file_zg_tot}")
        #print(f"file_zg_str = {file_zg_str}")
        
        
        file_zg_tot_train = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                            np.arange(yrst_train, yrst_train+yrnum_train_red+yrnum_test), invert=True))
        file_zg_tot_cv = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                            np.arange(yrst_train, yrst_train+yrnum_test), invert=False))             

        data_yr_reshaped_train = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                    np.arange(yrst_train, yrst_train+yrnum_train_red+yrnum_test), invert=True))
        data_yr_reshaped_cv = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                    np.arange(yrst_train, yrst_train+yrnum_test), invert=False))                
        
        if yrst_train == 2050:
            #remove backwards from 2016
            file_zg_tot_train = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                                np.arange(yrst_train-yrnum_train_red, yrst_train+yrnum_test), invert=True))
            file_zg_tot_cv = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                                np.arange(yrst_train, yrst_train+yrnum_test), invert=False))             

            data_yr_reshaped_train = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                        np.arange(yrst_train-yrnum_train_red, yrst_train+yrnum_test), invert=True))
            data_yr_reshaped_cv = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                        np.arange(yrst_train, yrst_train+yrnum_test), invert=False)) 
            
        if yrst_train-yrnum_train_red > 1960:
            #remove forwards from 1984
            file_zg_tot_train = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                                np.arange(yrst_train, yrst_train+yrnum_train_red+yrnum_test), invert=True))
            file_zg_tot_cv = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                                np.arange(yrst_train, yrst_train+yrnum_test), invert=False))             

            data_yr_reshaped_train = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                        np.arange(yrst_train, yrst_train+yrnum_train_red+yrnum_test), invert=True))
            data_yr_reshaped_cv = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                        np.arange(yrst_train, yrst_train+yrnum_test), invert=False))            
        
        if yrst_train == 1960:
            print(f"yrst_train == 1980")
            #remove forwards from 1984
            file_zg_tot_train = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                                np.arange(yrst_train, yrst_train+yrnum_train_red+yrnum_test), invert=True))
            file_zg_tot_cv = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                                np.arange(yrst_train, yrst_train+yrnum_test), invert=False))             

            data_yr_reshaped_train = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                        np.arange(yrst_train, yrst_train+yrnum_train_red+yrnum_test), invert=True))
            data_yr_reshaped_cv = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                        np.arange(yrst_train, yrst_train+yrnum_test), invert=False))        
        

            
    
        
        #for 1984-1987 and yrnum_train_red > 31
        yrnum_test = 10
        if yrst_train+yrnum_train_red+yrnum_test > 2060:
            max_yrs_train = 91
            num_years_to_train = max_yrs_train-yrnum_train_red
            print(f"num_years_to_train = {num_years_to_train}")
            #if you need to remove forwards and backwards
            file_zg_tot_train = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                                np.arange(yrst, yrst+num_years_to_train), invert=False))
            file_zg_tot_cv = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                                np.arange(yrst_train, yrst_train+yrnum_test), invert=False))             

            data_yr_reshaped_train = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                        np.arange(yrst, yrst+num_years_to_train), invert=False))
            data_yr_reshaped_cv = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                        np.arange(yrst_train, yrst_train+yrnum_test), invert=False))     
            
            
        if yrst_train-num_yrs_train < yrst:
            #can exclusively use years before start of test period for training
            file_zg_tot_train = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                                np.arange(yrst, yrst_train-num_yrs_train), invert=False))
            data_yr_reshaped_train = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                        np.arange(yrst, yrst_train-num_yrs_train), invert=False))
        elif yrst_train+yrnum_test+num_yrs_train < yrend:
            #can exclusively use years after end of test period for training            
            file_zg_tot_train = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                                np.arange(yrst_train+yrnum_test, yrst_train+yrnum_test+num_yrs_train), invert=False))
            data_yr_reshaped_train = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                        np.arange(yrst_train+yrnum_test, yrst_train+yrnum_test+num_yrs_train), invert=False))                
        else:
            #need to use years from both before and after the training period
            file_zg_tot_train_after = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                                np.arange(yrst_train+yrnum_test, yrst_train+yrnum_test+num_yrs_train), invert=False))
            data_yr_reshaped_train_after = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                        np.arange(yrst_train+yrnum_test, yrst_train+yrnum_test+num_yrs_train), invert=False))   
            num_yrs_training_after = len(np.unique(file_zg_tot_train_after['time.year'])) 
            print(f"num_yrs_training_after = {num_yrs_training_after}")
            print(f"years to add = {num_yrs_train-num_yrs_training_after}")
            
            yrs_to_add_before = num_yrs_train-num_yrs_training_after
            
            file_zg_tot_train = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], 
                                                np.arange(yrst+yrs_to_add_before, yrst_train+yrnum_test)#[1999,2000,2001,2002,2003,2004]
                                                               , invert=True))
            data_yr_reshaped_train = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time.year'], 
                                                       np.arange(yrst+yrs_to_add_before, yrst_train+yrnum_test), invert=True))               
        
        
        num_yrs_training = len(np.unique(file_zg_tot_train['time.year'])) 
        print(f"num_yrs_training = {num_yrs_training}")
        print(f"data_yr_reshaped_train = {np.unique(data_yr_reshaped_train['time.year'])}")
        print(f"data_yr_reshaped_cv = {np.unique(data_yr_reshaped_cv['time.year'])}") 
        print(f"file_zg_tot_train = {np.unique(file_zg_tot_train['time.year'])}")
        print(f"file_zg_tot_cv = {np.unique(file_zg_tot_cv['time.year'])}")        
        yrend_cv = yrst_train+yrnum_test-1
        if "UKESM" in file_zg_str:
            #/rds/general/user/cmt3718/home/data/cmip6/SOM/trends/UK-ESM1-0-LL_piControl/zg/crossval/10-fold
            #/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/zg/crossval/5-fold/
            savefig_SOM = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/training_yrs/plots/"
                   f"SOM_fig_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrend_cv)}_{season}_{zg_str}_LTDManom_{num_yrs_training}yrstrain.png")
            savefig_trends = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/training_yrs/plots/"
                   f"SOMtrends_fig_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrend_cv)}_{season}_{zg_str}_LTDManom_{num_yrs_training}yrstrain.png")
            SOM_trend_str_train = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/training_yrs/"
                   f"SOM_train_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrend_cv)}_{season}_{zg_str}_LTDManom{num_yrs_training}yrstrain.nc")
            SOM_data_occ_str_train = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/training_yrs/"
                   f"SOM_data_occur_train_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrend_cv)}_{season}_{zg_str}_LTDManom{num_yrs_training}yrstrain.nc")
            SOM_trend_str_cv = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/training_yrs/"
                   f"SOM_cv_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_{int(yrst_train)}-{int(yrend_cv)}_{season}_{zg_str}_LTDManom{num_yrs_training}yrstrain.nc")
            SOM_data_occ_str_cv = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/training_yrs/"
                   f"SOM_data_occur_cv_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_{int(yrst_train)}-{int(yrend_cv)}_{season}_{zg_str}_LTDManom{num_yrs_training}yrstrain.nc")

        elif "era5" in file_zg_str:
            savefig_SOM = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/training_yrs/plots/"
                   f"SOM_fig_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrend_cv)}_{season}_{zg_str}_LTDManom{num_yrs_training}yrstrain.png")
            savefig_trends = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/training_yrs/plots/"
                   f"SOMtrends_fig_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrend_cv)}_{season}_{zg_str}_LTDManom{num_yrs_training}yrstrain.png")
            SOM_trend_str_train = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/training_yrs/"
                   f"SOM_train_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrend_cv)}_{season}_{zg_str}_LTDManom{num_yrs_training}yrstrain.nc")
            SOM_data_occ_str_train = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/training_yrs/"
                   f"SOM_data_occur_train_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrend_cv)}_{season}_{zg_str}_LTDManom{num_yrs_training}yrstrain.nc")
            SOM_trend_str_cv = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/training_yrs/"
                   f"SOM_cv_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_{int(yrst_train)}-{int(yrend_cv)}_{season}_{zg_str}_LTDManom{num_yrs_training}yrstrain.nc")
            SOM_data_occ_str_cv = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/training_yrs/"
                   f"SOM_data_occur_cv_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_{int(yrst_train)}-{int(yrend_cv)}_{season}_{zg_str}_LTDManom{num_yrs_training}yrstrain.nc")
        else:
            print("not UKESM or era5 - cannot define save directories")
            return
        print(f"saving training in {SOM_trend_str_train}")
        print(f"saving cv in {SOM_trend_str_cv}")  
     
        JJA_days = int(len(np.unique(file_zg_tot['time.dayofyear'])))
        if "era5" in file_zg_str and "_z_" in file_zg_str:# and "pv_350pt" not in file_zg_str and "vort" not in file_zg_str:
            JJA_days = 100
            #JJA_days = JJA_days-1 #where there are leap years, need to remove from JJA_days since dayofyear will reflec the different leap year
        JJA_days = 98
        #print(f"JJA_days = {JJA_days}")
        #load the training dataset
        da_xr_trend_train, all_occur_train, som_yr_train = save_SOM_data(file_zg_tot_train, yrst, yrend, 
                    season, region, init, neigh, std, ep, rad_0, rad_N, rad_cooling, scale_0, scale_N, scale_cooling,
                    savefig_SOM, savefig_trends, n_rows, n_columns, samp, zg_str, colormap_str, suptitle, mult_test_method, 
                        data_yr_reshaped_train, vmin, vmax, step, lat_str, lon_str, units, mdl_ens_str, train_input = False, JJA_days=JJA_days, save_SOM_str=savefig_SOM)

        #print("use trained SOM to calculate cv dataset")
        #using the trained SOM, calculate the 
        da_xr_trend_cv, all_occur_cv, som_yr_train = save_SOM_data(file_zg_tot_cv, yrst, yrend, 
                    season, region, init, neigh, std, ep, rad_0, rad_N, rad_cooling, scale_0, scale_N, scale_cooling,
                    savefig_SOM, savefig_trends, n_rows, n_columns, samp, zg_str, colormap_str, suptitle, mult_test_method, 
                        data_yr_reshaped_cv, vmin, vmax, step, lat_str, lon_str, units, mdl_ens_str, train_input = True, som_yr = som_yr_train, JJA_days=JJA_days)


        da_xr_trend_train.to_netcdf(SOM_trend_str_train)
        da_train=xarray.DataArray(all_occur_train, name = "SOM_data", dims = ("row", "col", "time"))
        da_train['time']=file_zg_tot_train['time']
        da_train.to_netcdf(SOM_data_occ_str_train)
      
        da_xr_trend_cv.to_netcdf(SOM_trend_str_cv)
        da_cv=xarray.DataArray(all_occur_cv, name = "SOM_data", dims = ("row", "col", "time"))
        da_cv['time']=file_zg_tot_cv['time']
        da_cv.to_netcdf(SOM_data_occ_str_cv)
    return


In [1]:
# #test what years are missing from the SOM CV data which need to be run
# dir_str="/rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/crossval/10-fold/training_yrs/"


# year_str_arr = ["1980-1983","1984-1987","1988-1991","1992-1995","1996-1999","2000-2003","2004-2007","2008-2011","2012-2015","2016-2019"]


# for year_str in year_str_arr:
    
#     file_list=sorted(glob.glob(f"{dir_str}*SOM_train_era5_reanal_5x4_EUR_not{year_str}_JJA_extd_z_LTDManom*"))#3yrstrain.nc"))
#     years_done = sorted([int(file.split("LTDManom")[1].split("yrstrain")[0]) for file in file_list])
#     if len(file_list) != 37:
#         print(year_str)
#         print(years_done)
#         print(len(file_list))


In [9]:

#comment out the rest of the files dpeending on which you want to run
#UKESM files
file_zg_str = "/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/zg/500zg_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR_JJAextd_dtrnd.nc"
# file_zg_str = "/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/psl/psl_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR_JJAextd_dtrnd.nc"
# file_zg_str = "/rds/general/project/carl_phd/live/carl/data/cmip6/UKESM1-0-LL/piControl/day/r1i1p1f2/rel_vort/500xi_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_1960-2060_r180x91_latcorr_EUR_LTDManom_JJAextd.nc"


#ERA5 files
# file_zg_str = "/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_sort.nc"
# file_zg_str = "/rds/general/project/nowack_graven/live/carl/era5/mean_sea_level_pressure/mslp_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom.nc"
# file_zg_str = "/rds/general/user/cmt3718/home/pv_350pt_era5_1979-2019_1x1_tot_3hr_daymean_ydayavg_LTDManom_invlat_JJAextd.nc"
# file_zg_str = "/rds/general/project/carl_phd/live/carl/data/era5/day/pv/isentropes/pv_330pt_era5_1979-2019_1x1_EUR_daymean_invlat_LTDManom_JJAextd2.nc"
# file_zg_str = "/rds/general/project/carl_phd/live/carl/data/era5/day/rel_vort/500xi_era5_day_1979-2019_1x1_EUR_LTDManom_invlat.nc"
# file_zg_str = "/rds/general/project/nowack_graven/live/carl/pv_era5/vpv_1x1_day_150-500hPa_hires_1979-2019_LTDManom_JJAextd_EUR.nc"


#files todo:
###setup theta on PV=2

#"/rds/general/user/cmt3718/ephemeral/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/relvort/500xi_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_1960-2060_r180x91_latcorr_EUR_LTDManom_JJAextd_reshaped.nc"


###setup one other files for UKESM1-0-LL: vort anom 500hPa

if "era5" in file_zg_str:
    mip = "era5"
    yrnum_test = 3
    tot_num_years_10fold = 37
else:
    mip = "UKESM1-0-LL"
    yrnum_test = 10
    tot_num_years_10fold = 91
    
#yrnum_test = 10
yrnum_train_red = 0#if 0, train on 37 years, if 10 train on 27 years etc
rowcols_arr = [(2, 1), (3, 1), (2, 2), (5, 1), (3, 2), (7, 1), (4, 2), (3, 3), (5, 2), (11, 1), (4, 3), (13, 1), (7, 2), (5, 3), (4, 4), (17, 1), (6, 3),
 (19, 1), (5, 4), (7, 3), (11, 2), (23, 1), (6, 4), (5, 5), (13, 2), (9, 3), (7, 4), (29, 1), (6, 5), (31, 1), (8, 4), (33, 1), (17, 2),
               (7, 5), (6, 6), (37, 1), (19, 2), (39, 1), (8, 5), (41, 1)]

rowcols_arr = [(5, 4)]

for yrnum_train_red in range(,91):#range(10,91,10):#range(0,92):
	for nrow, ncol in rowcols_arr:
    		num_yrs_train = tot_num_years_10fold-yrnum_train_red
    		SOM_calc_samp(file_zg_str, nrow, ncol, yrnum_test, yrnum_train_red, num_yrs_train)

 --------------------- 
 
train period 1960-1970
yrst_train == 1980
num_yrs_training = 42
data_yr_reshaped_train = [2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031 2032
 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046
 2047 2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060]
data_yr_reshaped_cv = [1960 1961 1962 1963 1964 1965 1966 1967 1968 1969]
file_zg_tot_train = [2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031 2032
 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046
 2047 2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060]
file_zg_tot_cv = [1960 1961 1962 1963 1964 1965 1966 1967 1968 1969]
saving training in /rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/zg/crossval/10-fold/training_yrs/SOM_train_UKESM1-0-LL_r1i1p1f2_5x4_EUR_not1960-1969_JJA_extd_zg_LTDManom42yrstrain.nc
saving cv in /rds/general/project/nowack_graven/live/carl_som_index/data/U

KeyboardInterrupt: 